In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import re
import itertools

In [ ]:
tools_dir_path = os.path.abspath('')
root_dir_path = os.path.dirname(tools_dir_path)
print(root_dir_path)
data_dir_path = os.path.join(root_dir_path, 'results')

In [ ]:
def get_color(index):
    color_table = [
        
        "#27ae60","#2980b9","#8e44ad","#2c3e50",
        "#f1c40f","#e67e22","#e74c3c",
         "#d63031","#feca57","#5f27cd","#54a0ff","#01a3a4",
         "#16a085",
        "#25CCF7","#FD7272","#54a0ff","#00d2d3",
        "#1abc9c","#2ecc71","#3498db","#9b59b6","#34495e",
        # "#ecf0f1", "#bdc3c7", "#dfe6e9", 
        "#00b894","#00cec9","#0984e3","#6c5ce7","#ffeaa7",
        "#fab1a0","#ff7675","#fd79a8","#fdcb6e","#e17055",
        "#95a5a6","#f39c12","#d35400","#c0392b",
        "#55efc4","#81ecec","#74b9ff","#a29bfe",
        "#7f8c8d", # grey
        
       
    ]
    color_num = len(color_table)
    index %= color_num
    color_hex = color_table[index]
    color_int = int(color_hex.lstrip('#'),16)
    blue_int = color_int % 256
    color_int //= 256
    green_int = color_int % 256
    red_int = color_int // 256
    # return red_int, green_int, blue_int
    return (red_int / 256, green_int / 256, blue_int / 256)

In [ ]:
import matplotlib.ticker as plticker
from matplotlib.ticker import FormatStrFormatter
def plot_lines(title, x_arr_list, y_arr_list, label_list, x_axis_label, y_axis_label, use_log_xscale=True, use_log_yscale=False, show_fig=True):
    fig, ax = plt.subplots(figsize=(8, 6), dpi=300)
    fig.suptitle(title, fontsize='small')
    assert(len(x_arr_list) == len(y_arr_list))
    arr_num = len(x_arr_list)

    marker = itertools.cycle(('v','o','^','<','>','s','p','*','+','x','D'))
    for i in range(arr_num):
        color = get_color(i)
        ax.plot(x_arr_list[i], y_arr_list[i], label=label_list[i], color=color, marker=next(marker), markersize=2.5, linewidth=1.0)
    ax.set_xlabel(x_axis_label)
    ax.set_ylabel(y_axis_label)
    if use_log_xscale:
        ax.set_xscale('log')
    if use_log_yscale:
        ax.set_yscale('log')
    
    subs = [1.0, 2.0, 4.0, 6.0]
    ax.yaxis.set_minor_locator(plticker.LogLocator(subs=subs))

    box = ax.get_position()
    ax.set_position([box.x0, box.y0, box.width * 0.95, box.height])

    fig.legend(loc='center right', fontsize='xx-small')

    if show_fig:
        plt.show()
    plt.close(fig)
    

In [ ]:
import copy
latency_filename_list = ["flashws_latency.csv",  "tpp_latency.csv", "uws_poll_latency.csv", "uws_latency.csv"]
msg_size_arr_list = []
goodput_arr_list = []
p50_arr_list = []
p99_arr_list = []
test_item_name_list = []
for latency_filename in latency_filename_list:
    print(latency_filename)
    latency_file_path = os.path.join(data_dir_path, latency_filename)
    temp_df = pd.read_csv(latency_file_path, sep=',')
    # print(temp_df.head())
    msg_size_arr = temp_df["msg_size"].to_list()
    msg_size_arr_list.append(msg_size_arr)
    goodput_arr = np.array(temp_df["avg_goodput"].to_list())
    #make it Gbps
    goodput_arr /= 1000.0 
    goodput_arr_list.append(goodput_arr)
    p50_latency_arr = temp_df["P50_latency"].to_list()
    p50_arr_list.append(p50_latency_arr)
    p99_latency_arr = temp_df["P99_latency"].to_list()
    p99_arr_list.append(p99_latency_arr)
    m = re.findall(r'(.+)_latency.csv', latency_filename)
    # print(m)
    temp_test_item = m[0]
    test_item_name_list.append(temp_test_item)




all_latency_label_list = []
all_latency_arr_list = []
for i, test_item_name in enumerate(test_item_name_list):
    all_latency_label_list.append(test_item_name + "-P50")
    all_latency_arr_list.append(p50_arr_list[i])

    all_latency_label_list.append(test_item_name + "-P99")
    all_latency_arr_list.append(p99_arr_list[i])
# print(all_latency_label_list)
double_msg_size_arr_list = copy.deepcopy(msg_size_arr_list)
double_msg_size_arr_list.extend(msg_size_arr_list)
plot_lines("", double_msg_size_arr_list, all_latency_arr_list, all_latency_label_list, "Message Size (Byte)", "Latency (us)", use_log_xscale=True, use_log_yscale=True, show_fig=True)
plot_lines("", msg_size_arr_list, goodput_arr_list, test_item_name_list, "Message Size (Byte)", "Goodput (Gbps)", use_log_xscale=True, use_log_yscale=False, show_fig=True)

In [ ]:
throughput_filename_list = ["flashws_throughput.csv", "tpp_poll_throughput.csv", "uws_poll_throughput.csv"]

msg_size_arr_list = []
goodput_arr_list = []
test_item_name_list = []
msg_speed_arr_list = []

for throughput_name in throughput_filename_list:
    put_file_path = os.path.join(data_dir_path, throughput_name)
    temp_df = pd.read_csv(put_file_path, sep=',')
    msg_size_arr = temp_df["msg_size"].to_list()
    msg_size_arr_list.append(msg_size_arr)
    goodput_arr = np.array(temp_df["rx_goodput"].to_list())
    goodput_arr /= 1000.0 
    goodput_arr_list.append(goodput_arr)
    msg_speed_arr = temp_df["rx_mm_mps"].to_list()
    msg_speed_arr_list.append(msg_speed_arr)
    connection_cnt_arr = temp_df['connection_cnt'].to_list()

    m = re.findall(r'(.+)_throughput.csv', throughput_name)
    # print(m)
    temp_test_item = m[0]
    test_item_name_list.append(temp_test_item)

plot_lines("", msg_size_arr_list, goodput_arr_list, test_item_name_list, "Message Size (Byte)", "Goodput (Gbps)", use_log_xscale=True, use_log_yscale=False, show_fig=True)
plot_lines("", msg_size_arr_list, msg_speed_arr_list, test_item_name_list, "Message Size (Byte)", "Messages/sec (Mpps)", use_log_xscale=True, use_log_yscale=False, show_fig=True)